In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets, utils
from torch.autograd import Variable

torch.manual_seed(42)

# Step 1 Build data pipeline and import image dataset (split into training set and test set 5:5)

In [28]:
# data_dir is the address where you store your data data, I put it here under relative path
data_dir = "D:/zhuomian/2022_project/data"
split = 0.5
batch_size = 32
learning_rate = 0.001
num_epochs = 30

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)

device: cpu


In [29]:
# Build data pre-processing
data_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Resize((56,56)),
                                     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

# Import data into dataset
dataset = datasets.ImageFolder(root = data_dir, transform = data_transform)

# Now divide the data set into a training set and a test set
len_imgs = len(dataset.imgs)
train_set, test_set = torch.utils.data.random_split(dataset, [int(len_imgs*split), len_imgs-int(len_imgs*split)]) 

# Final construction of the data pipeline
train_loader = torch.utils.data.DataLoader(train_set,batch_size = batch_size,shuffle = True)
test_loader = torch.utils.data.DataLoader(test_set,batch_size = batch_size,shuffle = False)

# Step 2 Build the model Customized loss function and optimizer

In [21]:
class CNN_model(nn.Module):
    def __init__(self):
        super(CNN_model, self).__init__()
        # conv2d Parameters nn.Conv2d(in_channels=3, out_channels=32,kernel_size=5,stride=1,padding="same")
        # MaxPool2d Parameters nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv0 = nn.Sequential(         
            nn.Conv2d(32, 48, 1, 1, padding = "same"),
            nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.Conv2d(48, 48, 3, 1, padding = "same"),
            nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.Conv2d(48, 32, 1, 1, padding = "same"),
            nn.BatchNorm2d(32),
            nn.ReLU(),
        )
        
        self.conv1 = nn.Sequential(         
            nn.Conv2d(3, 32, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2) 
        )
        
        self.conv2 = nn.Sequential(         
            nn.Conv2d(32, 64, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2)                
        )
        
        self.conv3 = nn.Sequential(         
            nn.Conv2d(64, 32, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2)                
        )
        
        # Fully connected layer with 21 output categories
        self.fc1 = nn.Sequential(nn.Linear(32 * 7 * 7, 128),nn.ReLU())
        self.fc2 = nn.Sequential(nn.Linear(128, 21))
    def forward(self, x):
        x = self.conv1(x)
        identity = x
        x = self.conv0(x)
        x = identity + x
        x = self.conv2(x)
        x = self.conv3(x)
        # Flattening treatment x.view
        x = x.view(x.size(0), -1)       
        x = self.fc1(x)
        output = self.fc2(x)
        return output    # return x for visualization

In [22]:
model = CNN_model()
model.to(device)
# Choose whether to print the model or not, here it will not be printed, just comment
print(model)

# Select loss function and optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),
                       lr=learning_rate, betas=(0.9, 0.999),
                       eps=1e-07, amsgrad=False)

CNN_model(
  (conv0): Sequential(
    (0): Conv2d(32, 48, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (4): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (7): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
  )
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0

# Step 3 Pass in the model to start training and testing

In [23]:
# Train the model and record the accuracy and loss values for each epoch during training
def train(num_epochs, model, loaders):
    model.train()   
    
    # The first stores the correct rate and the second stores the loss value
    Accy_list = []
    Loss_list = []
    
    total_step = len(loaders)    
    for epoch in range(num_epochs):
        correct = 0
        L = 0
        for i, (images, labels) in enumerate(loaders):
            images = images.to(device)
            labels = labels.to(device)              
            outputs = model(images)
            loss = loss_func(outputs, labels)
            
            # Clear the previous gradient 
            optimizer.zero_grad()           
            
            # Backpropagate and update the parameters
            loss.backward()               
            optimizer.step()                
            
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).float().sum()
            L += loss.item()
            if (i+1) % 9 == 0:
                print ('Epoch: [{}/{}]\t|\tStep: [{}/{}]\t|\tLoss: {:.4f}' 
                       .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))
                
        accuracy = 100 * correct / len(train_set)
        print(accuracy.item())
        print("epoch: {:02d} | Accuracy: {:.2f} Loss: {:.4f}\n"
              .format(epoch + 1, accuracy, L))
        
        # Add the correct and lost values for this epoch to the corresponding list
        Accy_list.append(accuracy.item())
        Loss_list.append(L)
              
    return Accy_list, Loss_list


Accy_list, Loss_list = train(num_epochs, model, train_loader)

Epoch: [1/30]	|	Step: [9/27]	|	Loss: 3.0176
Epoch: [1/30]	|	Step: [18/27]	|	Loss: 3.0666
Epoch: [1/30]	|	Step: [27/27]	|	Loss: 2.8864
7.015457630157471
epoch: 01 | Accuracy: 7.02 Loss: 81.4627

Epoch: [2/30]	|	Step: [9/27]	|	Loss: 2.7688
Epoch: [2/30]	|	Step: [18/27]	|	Loss: 2.5351
Epoch: [2/30]	|	Step: [27/27]	|	Loss: 2.1137
21.403091430664062
epoch: 02 | Accuracy: 21.40 Loss: 70.9568

Epoch: [3/30]	|	Step: [9/27]	|	Loss: 2.1728
Epoch: [3/30]	|	Step: [18/27]	|	Loss: 2.1713
Epoch: [3/30]	|	Step: [27/27]	|	Loss: 1.6618
37.21759796142578
epoch: 03 | Accuracy: 37.22 Loss: 56.3869

Epoch: [4/30]	|	Step: [9/27]	|	Loss: 1.9207
Epoch: [4/30]	|	Step: [18/27]	|	Loss: 1.3236
Epoch: [4/30]	|	Step: [27/27]	|	Loss: 1.3496
53.98335266113281
epoch: 04 | Accuracy: 53.98 Loss: 41.0286

Epoch: [5/30]	|	Step: [9/27]	|	Loss: 0.5564
Epoch: [5/30]	|	Step: [18/27]	|	Loss: 1.2274
Epoch: [5/30]	|	Step: [27/27]	|	Loss: 0.4992
70.63020324707031
epoch: 05 | Accuracy: 70.63 Loss: 26.0076

Epoch: [6/30]	|	Step: [9/

In [24]:
print(Accy_list)

[7.015457630157471, 21.403091430664062, 37.21759796142578, 53.98335266113281, 70.63020324707031, 79.54815673828125, 85.96908569335938, 92.86563873291016, 96.31391143798828, 98.33531188964844, 95.83828735351562, 97.74078369140625, 98.69203186035156, 99.40547180175781, 99.64328002929688, 99.7621841430664, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]


In [25]:
print(Loss_list)

[81.46269178390503, 70.95676946640015, 56.386890053749084, 41.02861762046814, 26.0075703561306, 16.398088335990906, 11.516255736351013, 5.973637692630291, 3.197179399430752, 1.8383868867531419, 3.872355970554054, 1.8238596199080348, 1.7378284595906734, 0.9488751759636216, 0.5671486143255606, 0.36874577967682853, 0.06439036119263619, 0.030855718738166615, 0.023343778389971703, 0.01659781878697686, 0.014191708964062855, 0.012824622841435485, 0.012078466010279953, 0.01076020544860512, 0.010739671815827023, 0.0093796493820264, 0.008752797308261506, 0.008637119659397285, 0.007778852101182565, 0.006469015796028543]


In [26]:
# Test models
def test(model, loaders):
    model.eval()
    with torch.no_grad():
        correct = 0
        L = 0
        for images, labels in loaders:
            images = images.to(device)
            labels = labels.to(device)              
            outputs = model(images)
            # Get the loss value and add it up
            loss = loss_func(outputs, labels)
            L += loss.item()
            
             # Get the correct number of predicted samples
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).float().sum()
            
    accuracy = (100 * correct / len(test_set))
    print("Test data | Accuracy: {:.2f} %, Loss: {:.4f} ".format(accuracy, L))
    
    return accuracy, L

Test_acc, Test_loss = test(model, test_loader)

Test data | Accuracy: 77.05 %, Loss: 43.7253 
